# To use this notebook

-   Open in Azure Data Studio
-   Ensure the Kernel is set to "PowerShell"

# You can run Flyway in a variety of ways

Community edition is free

You may download and install locally - [https://flywaydb.org/download/](https://flywaydb.org/download/)

You may use the flyway docker container - [https://github.com/flyway/flyway-docker](https://github.com/flyway/flyway-docker)

# Running the Flyway Docker container

Install Docker and make sure it's running - [https://docs.docker.com/get-docker/](https://docs.docker.com/get-docker/)

Instructions to run Flyway via Docker are here - [https://github.com/flyway/flyway-docker](https://github.com/flyway/flyway-docker)

Some examples of this are below

# Run Flyway and return info on available commands

If the image isn't available for you locally yet (first run), this command should automatically pull it.

The --rm causes Docker to automatically remove the container when it exits.

In [1]:
docker run --rm flyway/flyway

Usage
=====

flyway [options] command

By default, the configuration will be read from conf/flyway.conf.
Options passed from the command-line override the configuration.

Commands
--------
migrate  : Migrates the database
clean    : Drops all objects in the configured schemas
info     : Prints the information about applied, current and pending migrations
validate : Validates the applied migrations against the ones on the classpath
undo     : [teams] Undoes the most recently applied versioned migration
baseline : Baselines an existing database at the baselineVersion
repair   : Repairs the schema history table

Options (Format: -key=value)
-------
driver                       : Fully qualified classname of the JDBC driver
url                          : Jdbc url to use to connect to the database
user                         : User to use to connect to the database
password                     : Password to use to connect to the database
connectRetries               : Maximum number of ret

# A simple test of Flyway's info command using the H2 in memory database

In [2]:
docker run --rm flyway/flyway -url=jdbc:h2:mem:test -user=sa info

Flyway Community Edition 7.0.3 by Redgate
Database: jdbc:h2:mem:test (H2 1.4)
Schema version: << Empty Schema >>

+----------+---------+-------------+------+--------------+-------+
| Category | Version | Description | Type | Installed On | State |
+----------+---------+-------------+------+--------------+-------+
| No migrations found                                            |
+----------+---------+-------------+------+--------------+-------+



# Let's talk to a SQL Server

I'm using a config file here, by passing in a volume with -v. We are naming the volume /flyway/conf.

-   This needs to be an absolute path to the folder where you have flyway.conf
-   You will need to edit the connection string, user, and password in flyway.conf
-   You will need to create a database named GitForDBAs (or change the config file to reference a database of another name which already exists)

I'm using a second volume mapping to a folder that holds my flyway migrations. We are naming the volume /flyway/sql.

-   This needs to be an absolute path to the folder where you have migrations stored
-   The filenames for the migrations matter -- Flyway uses the file names to understand what type of script it is and the order in which it should be run

Note: I have spread this across multiple lines using the \` character for readability purposes  

# Call Flyway info to inspect

In [4]:
docker run --rm `
    -v C:\Git\GitForDBAs\flywayconf:/flyway/conf  `
    -v C:\Git\GitForDBAs\migrations:/flyway/sql  `
    flyway/flyway info

Flyway Community Edition 7.0.3 by Redgate
Database: jdbc:jtds:sqlserver://host.docker.internal:1433/GitForDBAs (Microsoft SQL Server 14.0)
Schema version: << Empty Schema >>

+-----------+---------+-----------------------+------+--------------+---------+
| Category  | Version | Description           | Type | Installed On | State   |
+-----------+---------+-----------------------+------+--------------+---------+
| Versioned | 1       | Initial               | SQL  |              | Pending |
| Versioned | 2       | YOLO                  | SQL  |              | Pending |
| Versioned | 2.1     | ILikeDags             | SQL  |              | Pending |
| Versioned | 2.2     | InsertRowsInILikeDags | SQL  |              | Pending |
| Versioned | 2.3     | livedemo              | SQL  |              | Pending |
+-----------+---------+-----------------------+------+--------------+---------+



#  Call Flyway migrate to execute

In [5]:
docker run --rm `
    -v C:\Git\GitForDBAs\flywayconf:/flyway/conf  `
    -v C:\Git\GitForDBAs\migrations:/flyway/sql  `
    flyway/flyway migrate

Flyway Community Edition 7.0.3 by Redgate
Database: jdbc:jtds:sqlserver://host.docker.internal:1433/GitForDBAs (Microsoft SQL Server 14.0)
Successfully validated 5 migrations (execution time 00:00.035s)
Creating Schema History table [GitForDBAs].[dbo].[flyway_schema_history] ...
Current version of schema [dbo]: << Empty Schema >>
Migrating schema [dbo] to version "1 - Initial"
Migrating schema [dbo] to version "2 - YOLO"
Migrating schema [dbo] to version "2.1 - ILikeDags"
Migrating schema [dbo] to version "2.2 - InsertRowsInILikeDags"
Migrating schema [dbo] to version "2.3 - livedemo"
Successfully applied 5 migrations to schema [dbo] (execution time 00:00.235s)


# Examine the table - open a new query

USE GitForDBAs;

GO

  

EXEC sp\_help 'dbo.HelloWorld';

GO

  

SELECT \*  FROM dbo.HelloWorld;

GO

# Call Flyway clean to drop everything 🔥🔥🔥

In [3]:
docker run --rm `
    -v C:\Git\GitForDBAs\flywayconf:/flyway/conf  `
    -v C:\Git\GitForDBAs\migrations:/flyway/sql  `
    flyway/flyway clean

Flyway Community Edition 7.0.3 by Redgate
Database: jdbc:jtds:sqlserver://host.docker.internal:1433/GitForDBAs (Microsoft SQL Server 14.0)
Successfully dropped pre-schema database level objects (execution time 00:00.002s)
Successfully cleaned schema [dbo] (execution time 00:00.177s)
Successfully dropped post-schema database level objects (execution time 00:00.007s)
